In [1]:
import pandas as pd
from math import radians, sin, cos, sqrt, atan2
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bruno\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [38]:
cat_g = pd.read_parquet('../../datasets/processed/bd/7_categories_google.parquet.gz')
bus_g = pd.read_parquet('../../datasets/processed/bd/5_business_google.parquet.gz')
cat_y = pd.read_parquet('../../datasets/processed/bd/8_categories_yelp.parquet.gz')
bus_y = pd.read_parquet('../../datasets/processed/bd/6_business_yelp.parquet.gz')
cate = pd.read_parquet('../../datasets/processed/bd/2_categories.parquet.gz')
loccat = pd.read_parquet('./datasets/locales_categories.parquet')
cat_g.rename(columns=({'gmap_id':'business_id'}),inplace=True)
cat_ge = pd.concat([cat_g,cat_y])

In [40]:
with open('./modelo_knn.pkl', 'rb') as file: 
    knn_model = pickle.load(file)
        
with open('./tfidf_matrix.pkl', 'rb') as file:
    categories_procceced = pickle.load(file)

C:\Users\bruno\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator NearestNeighbors from version 1.3.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [41]:
idx = loccat[loccat['business_id'] == 'i6COOaJ3doNtXJuiSvDzzA'].index[0]
_, indices = knn_model.kneighbors(categories_procceced[idx])
recommendations = loccat['business_id'].iloc[indices[0,1:]]  # Excluye el propio restaurante

In [29]:
def process_text(text):
    """
    Funcion que realiza tokeinizacion en base a un texto.

    Args:
        text (string): Palabra u oración para aplicar la tokeinizacin.

    Returns:
        str: Serie de strings.
    """
    # Aplico la teokeinizacion
    stop_words = set(stopwords.words('english'))
    ps = PorterStemmer()
    words = word_tokenize(text)
    words = [ps.stem(word.lower()) for word in words if word.isalnum() and word.lower() not in stop_words]
    return ' '.join(words)

In [23]:
cat_ge = pd.merge(cat_ge,cate,on='categories_id',how='inner')

In [30]:
cat_ge['procceced'] = cat_ge['name'].apply(process_text)

# Si hay mas clase ademas de restaur ej: pizza restaur borra restaur, si no deja igual
cat_ge['procceced'] = cat_ge['procceced'].apply(lambda x:x.replace('restaur','') if x!= 'restaur' else x)
cat_ge['procceced'] = cat_ge['procceced'].astype(str)

In [33]:
def obtener_palabras_similares(palabra, modelo, topn=3):
    try:
        similares = modelo.similar_by_word(palabra, topn=topn)
        return [palabra for palabra, _ in similares]
    except KeyError:
        return []

In [34]:
from gensim.models import KeyedVectors

# Ruta al archivo GoogleNews-vectors-negative300.bin
ruta_modelo = '../../datasets/extras/model/GoogleNews-vectors-negative300.bin/GoogleNews-vectors-negative300.bin'

# Cargar el modelo
modelo = KeyedVectors.load_word2vec_format(ruta_modelo, binary=True,limit=1000)
    
    
cat_ge['processed'] = cat_ge['procceced'].apply(
lambda text: ' '.join(
    [
        ' '.join(obtener_palabras_similares(palabra.strip(), modelo)) 
        if palabra in text 
        else palabra 
        for palabra in text.split()
    ]
)
)   

In [35]:
cat_ge

,business_id,categories_id,name,procceced,processed
0,0x88d9ba5d65937567:0xbc27649cf513cc89,0,restaurant,restaur,
1,0x88d900575f0dd065:0x9b3638d2a80be4d,0,restaurant,restaur,
2,0x88d9b719170a9f61:0x30d454a980f76ad3,0,restaurant,restaur,
3,0x88e76652cd84272f:0x548abb9935d912ff,0,restaurant,restaur,
4,0x88d9b395c6179c89:0x914b0aafb453b3b5,0,restaurant,restaur,
...,...,...,...,...,...
132590,vV57YWbrHqm1iylWmIdwVA,1292,botanical gardens,botan garden,
132591,y7P06O7ypUgdU5fgV4s87Q,1293,newspapers & magazines,newspap magazin,
132592,jchZL8NJP9YBwgFmrSdpfQ,1294,parking,park,
132593,9295K2aKOltQSSTpm_7DBQ,1295,georgian,georgian,


### Usando nltk para procesar las categorias, y 

In [10]:

def process_text(text):
    """
    Funcion que realiza tokeinizacion en base a un texto.

    Args:
        text (string): Palabra u oración para aplicar la tokeinizacin.

    Returns:
        str: Serie de strings.
    """
    # Aplico la teokeinizacion
    stop_words = set(stopwords.words('english'))
    ps = PorterStemmer()
    words = word_tokenize(text)
    words = [ps.stem(word.lower()) for word in words if word.isalnum() and word.lower() not in stop_words]
    return ' '.join(words)



def obtener_palabras_similares(palabra, modelo, topn=3):
    try:
        similares = modelo.similar_by_word(palabra, topn=topn)
        return [palabra for palabra, _ in similares]
    except KeyError:
        return []

def categories_nlp():  
    """
    Funcion que a partir de un dataframe con categorias de columna "name" aplica la funcion *process_text*

    Returns:
        TfidfVectorizer: Matriz TF-IDF represetando  la frecuencia de terminos ponderada por importancia.(necesaria para el modelo.)
    """
    
    
    
    #Genero un dataframe que contenga, las categorias y los negocios para yelp y google.

    # Cambiar por la lectura a la BD

    local_categories_google = pd.read_parquet('../../datasets/processed/bd/7_categories_google.parquet.gz')

    # Cambiar por la lectura a la BD
    local_categories_yelp = pd.read_parquet('../../datasets/processed/bd/8_categories_yelp.parquet.gz')

    #Si se lee de la base de datos business_id ya esta como nombre.
    local_categories_google.rename(columns={'gmap_id':'business_id'},inplace=True)
    local_categories = pd.concat([local_categories_google,local_categories_yelp])

    # Cambiar por la lectura a la BD
    categoires = pd.read_parquet('../../datasets/processed/bd/2_categories.parquet.gz')
    local_categories = pd.merge(local_categories,categoires,on='categories_id',how='inner')
    
    #### Se genera el dataframe local_categories.#####
    
    
    
    local_categories['procceced'] = local_categories['name'].apply(process_text)

    # Si hay mas clase ademas de restaur ej: pizza restaur borra restaur, si no deja igual
    local_categories['procceced'] = local_categories['procceced'].apply(lambda x:x.replace('restaur','') if x!= 'restaur' else x)
    local_categories['procceced'] = local_categories['procceced'].astype(str)
    # Crear una matriz TF-IDF para medir la similitud del contenido
    
    from gensim.models import KeyedVectors

    # Ruta al archivo GoogleNews-vectors-negative300.bin
    ruta_modelo = '../../datasets/extras/model/GoogleNews-vectors-negative300.bin/GoogleNews-vectors-negative300.bin'

    # Cargar el modelo
    modelo = KeyedVectors.load_word2vec_format(ruta_modelo, binary=True,limit=500000)
    
    
    local_categories['processed'] = local_categories['procceced'].apply(
    lambda text: ' '.join(
        [
            ' '.join(obtener_palabras_similares(palabra.strip(), modelo)) 
            if palabra in text 
            else palabra 
            for palabra in text.split()
        ]
    )
    )   
    
    local_categories = local_categories[['business_id','name','processed']]
    local_categories.to_parquet('./datasets/locales_categories.parquet') # Guardo el dataset util


    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(local_categories['processed'])
    return tfidf_matrix

    #Proceso para normalizar las categorias

## Gnero las categorias para luego procesarlas y guardarlas

In [ ]:
#Genero un dataframe que contenga, las categorias y los negocios para yelp y google.

# Cambiar por la lectura a la BD

local_categories_google = pd.read_parquet('../../datasets/processed/bd/7_categories_google.parquet.gz')

# Cambiar por la lectura a la BD
local_categories_yelp = pd.read_parquet('../../datasets/processed/bd/8_categories_yelp.parquet.gz')

#Si se lee de la base de datos business_id ya esta como nombre.
local_categories_google.rename(columns={'gmap_id':'business_id'},inplace=True)
local_categories = pd.concat([local_categories_google,local_categories_yelp])

# Cambiar por la lectura a la BD
categoires = pd.read_parquet('../../datasets/processed/bd/2_categories.parquet.gz')
local_categories = pd.merge(local_categories,categoires,on='categories_id',how='inner')





In [ ]:
    
# Hago el procesamiento de las categorias con NLTK y los exporto en un pkl
categories_procceced = categories_nlp() 
with open('./tfidf_matrix.pkl', 'wb') as file:
        pickle.dump(categories_procceced, file)

In [ ]:
#EVALUACION DEL MODELO


# Cargo la matriz generada del procesamiento
with open('./tfidf_matrix.pkl', 'rb') as file:
        tfidf_matrix = pickle.load(file)
        
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(tfidf_matrix, test_size=0.2, random_state=42)

knn_model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=30)

knn_model.fit(tfidf_matrix)
local_categories = pd.read_parquet('./app/ml/datasets/locales_categories.parquet')
idx = local_categories[local_categories['business_id'] == business_id].index[0]

_, indices = knn_model.kneighbors(categories_procceced[idx])
recommendations = local_categories['business_id'].iloc[indices[0][1:]]


In [200]:
#Modelo de recomendacion usando similitudes con vecinos cercanos


# Cargo la matriz generada del procesamiento
with open('./tfidf_matrix.pkl', 'rb') as file:
        tfidf_matrix = pickle.load(file)

from sklearn.model_selection import train_test_split

#Defino y entreno al modelo.
knn_model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=30)
knn_model.fit(tfidf_matrix)

# Guardo el modelo en un pkl
with open('./modelo_knn.pkl', 'wb') as file:
    pickle.dump(knn_model, file)



FILTRO POR DISTANCIA

In [43]:
# Funcion que calcula la distancia entre dos punto en funcion de las coordenadas
def haversine(lat1, lon1, lat2, lon2):
    
    """
    Esta funcion aplica la distancia hervesine para encontrar la distancia entre dos puntos a partir de sus coordenaadas.
    
    Args:
        lat1 (float): Latitud del primer punto.
        lon1 (float): Longitud del primer punto
        lat2 (float: Latitud del segundo punto.
        lon2 (float): Longitud del segundo punto.
        
    Returns:
        float:Distancia en metros entre dos coordeandas.
    """
    
    # Radio de la Tierra en kilometros
    R = 6371.0

    # Convierte las coordenadas de grados a radianes
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Diferencia de latitud y longitud
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Fórmula haversine
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # Distancia en metros
    distance = R * c

    return distance

In [44]:
#Funcion que a partir de un id de negocio y una lista ids retorna la distancia entre ese negocio y cada uno de los demas
def distance(business_id,business_id_list,rang=None):
    
    """
    Esta funcion calcula a partir de un negocio y una lista de negocios, la distancia entre los puntos.

    Args:
        business_id(str): Id del negocio.
        business_id_list(list): Lista con id de negocios.
        rang(float,optional) : Maxima distancia(kilometros) sobre la cual se quiere devolver los negocios.

    Returns:
        pd.DataFrame: Data frame de los negocios tomando en cuenta las distnacias
    """
    
    if rang:
        filtro_distance = rang
    else:   
        filtro_distance = 300000000 #FIltro distancia en metros.
    #Genero un dataframe con los restaurantes de google y yelp
    
    # Cambiar por la lectura a la BD, si se lee de ahi business_id ya esta como nombre
    business_google=pd.read_parquet('../../datasets/processed/bd/5_business_google.parquet.gz') 
    business_google.rename(columns={'gmap_id':'business_id'},inplace=True) 
    
    # Cambiar por la lectura a la BD
    business_yelp=pd.read_parquet('../../datasets/processed/bd/6_business_yelp.parquet.gz') 
    
    # si se lee de la base de datos cambiar stars de business_yelp por avg_stars
    business = pd.concat([business_google[['business_id','name','avg_stars','latitude','longitude','state_id']],business_yelp[['business_id','name','avg_stars','latitude','longitude','state_id']]])
    #Genero las coordenadas del local al que le quiero encontrar recomendaciones.
    lat_origin,long_origin = business[business['business_id']==business_id]['latitude'].iloc[0],business[business['business_id']==business_id]['longitude'].iloc[0]
    #Filtro solo por los restaurantes que pertenecen a las recomendaciones.
    business = business[business['business_id'].isin(business_id_list)]
    #Calculo la distancia de cada restuarante recomendado al inicial
    business['distance'] = business.apply(lambda row: haversine(lat_origin, long_origin, row['latitude'], row['longitude']), axis=1)
    #Aplico el filtro de distancia.
    business = business[business['distance']<filtro_distance]
    return business
    
    

In [48]:
# Función para obtener recomendaciones
def get_recommendations(business_id,rang=None):
    
    """
    Funcion que a partir de un negocio, recomienda otros, en funcion de sus categorias usando el modelo KNN.
    
    Ags:
        business_id(str) : id del negocio al cual se le quieren calcular recomendaciones.
        rang(float,optional) :Rango de distancia para obtener las recomendaciones(kilometros).

    Returns:
        business_cat(pd.DataFrame):Data Frame con las recomendaciones junto no algunas caracteristicas del negocio.
    """
    
    # Cargo el modelo
    with open('./modelo_knn.pkl', 'rb') as file: 
        knn_model = pickle.load(file)
        
    with open('./tfidf_matrix.pkl', 'rb') as file:
        categories_procceced = pickle.load(file)
    
    ######### categories_procceced podria ser un df importado  con todos los pasos anteriores.#########
    local_categories = pd.read_parquet('./datasets/locales_categories.parquet')
    

    idx = local_categories[local_categories['business_id'] == business_id].index[0]
   
    
    
    #Genero las recomendaciones.
    _, indices = knn_model.kneighbors(categories_procceced[idx])
    print(indices)
    recommendations = local_categories['business_id'].iloc[indices[0][0:]]  # Excluye el propio restaurante
    
    #Calcula las distancias entre las recomendaciones y el local.
    if rang:
        business = distance(business_id,recommendations,rang)
    else:
        business = distance(business_id,recommendations)
    business = business[business['distance']!=0.0] # Elimino al restaurante mismos.
    #Uno las caractereisticas de los locales, con las categorias.
    business_cat = pd.merge(local_categories,business,on='business_id')
    business_cat = business_cat.groupby('business_id').agg({
        'latitude':'first',
        'longitude':'first',
        'name_x':list,
        'name_y':'first',
        'distance':'first',
        'avg_stars':'mean',
        'state_id':'first'
        
    }).reset_index().rename(columns =({'name_x':'category','name_y':'name'}))
        
    
    return business_cat


## FILTRO POR USUARIOS

In [46]:
#Funcion que recibe un business id userid o categoria y recomienda locales, tambien puede agregarse el rango en metros de distancia.
def recommendation(business_ids=None,user_id=None,category=None,distance=None,state=None):
    """
    Esta funcion a partird e un negocio usuario o categoria recomienda otros negocios, teniendo en cuenta la distancia de ser requerida.
    Para esto la funcion toma un negocio, o selecciona una lista de ellos usando user_id, y categorias, y aplica la funcion *get_recommendations*

    Args:
        business_ids (str, optional): Id de un negocio.
        user_id (str, optional): Id de un usuario.
        category (str, optional): Categoria (nombre).
        distance (float, optional): Distancia en kilometros.

    Returns:
        pd.DataFrame: Data Frame con ñas recomendaciones y otras caracteristicas(analizar el uso de json)
    """
        
    if business_ids:
        business_ids = [business_ids]
    
    if user_id:
        
        # Cambiar por la lectura a la BD
        df_rg = pd.read_parquet('../../datasets/processed/bd/9_reviews_google.parquet.gz',columns=['user_id','gmap_id','sentiment'])
        df_ry = pd.read_parquet('../../datasets/processed/bd/10_reviews_yelp.parquet.gz',columns=['user_id','business_id','sentiment'])
        df = pd.concat([df_rg,df_ry])
        business_ids = df[df['user_id']==user_id].iloc[:10]['business_id'].tolist()
        distance = None
        if len(business_ids) == 0:
            return 'Usuario no encontrado.'
        
    if category:
        df_categories = pd.read_parquet('./datasets/locales_categories.parquet')
        business_ids = df_categories[df_categories['name'].str.lower().str.contains(category.lower())].sample(10).iloc[:10]['business_id'].tolist()
        distance = None
        if len(business_ids) == 0:
            return 'Categoria no encontrada.'
        
        
    business_cat = pd.DataFrame()
    
    for business_id in business_ids:
        business_cat = pd.concat([get_recommendations(business_id,rang=distance),business_cat])    
        
    if business_cat.shape[0] == 0:
        return 'Restaurante no encontrado.'
    
    states = pd.read_parquet('../../datasets/processed/bd/1_states.parquet.gz')
    business_cat = pd.merge(business_cat,states,on='state_id',how='inner')
    business_cat = business_cat[['business_id','name','category','state','latitude','longitude','avg_stars','distance']]
    if state:
        business_cat = business_cat[business_cat['state']==state]
        
    return business_cat.sort_values(by=['distance','avg_stars'],ascending=[True,False]).iloc[0:10]

In [49]:
recommendation(business_ids='0x88d9b395c6179c89:0x914b0aafb453b3b5')

C:\Users\bruno\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator NearestNeighbors from version 1.3.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


[[172054 172055 172028 172053 172052 172051 172050 172049 172048 172047
  172046 172045 172044 172043 172042 172041 172040 172039 172038 172037
  172036 172035 172034 172033 172032 172031 172030 172056  54821  54822]]


IndexError: positional indexers are out-of-bounds

In [18]:
df_categories = pd.read_parquet('./datasets/locales_categories.parquet')
df_categories

,business_id,name,processed
0,0x88d9ba5d65937567:0xbc27649cf513cc89,restaurant,
1,0x88d900575f0dd065:0x9b3638d2a80be4d,restaurant,
2,0x88d9b719170a9f61:0x30d454a980f76ad3,restaurant,
3,0x88e76652cd84272f:0x548abb9935d912ff,restaurant,
4,0x88d9b395c6179c89:0x914b0aafb453b3b5,restaurant,
...,...,...,...
132590,vV57YWbrHqm1iylWmIdwVA,botanical gardens,gardens vegetable_garden herb_garden
132591,y7P06O7ypUgdU5fgV4s87Q,newspapers & magazines,
132592,jchZL8NJP9YBwgFmrSdpfQ,parking,parks Park skate_park
132593,9295K2aKOltQSSTpm_7DBQ,georgian,


In [15]:
recommendation(business_ids='pizza',distance=100000.0)

IndexError: index 0 is out of bounds for axis 0 with size 0

# AHORA INTENTAMOS CON WORDVEC

In [ ]:
from gensim.models import KeyedVectors

# Ruta al archivo GoogleNews-vectors-negative300.bin
ruta_modelo = '../../datasets/extras/model/GoogleNews-vectors-negative300.bin/GoogleNews-vectors-negative300.bin'

# Cargar el modelo
modelo = KeyedVectors.load_word2vec_format(ruta_modelo, binary=True,limit=500000)



In [ ]:
def obtener_palabras_similares(palabra, modelo, topn=3):
    try:
        similares = modelo.similar_by_word(palabra, topn=topn)
        return [palabra for palabra, _ in similares]
    except KeyError:
        return []

In [ ]:
print(obtener_palabras_similares('restaurant',modelo))

In [ ]:
local_categories.sample(3)

In [ ]:
local_categories['processed'] = local_categories['procceced'].apply(
    lambda text: ' '.join(
        [
            ' '.join(obtener_palabras_similares(palabra.strip(), modelo)) 
            if palabra in text 
            else palabra 
            for palabra in text.split()
        ]
    )
)

In [ ]:
local_categories.loc[local_categories['processed'] == '', 'processed'] = 'restaur'

In [ ]:
local_categories.sample(5)

In [ ]:



tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(local_categories['processed'])



#Defino y entreno al modelo.
knn_model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=10)
knn_model.fit(tfidf_matrix)

# Guardo el modelo en un pkl
with open('./model/modelo_knn.pkl', 'wb') as file:
    pickle.dump(knn_model, file)

In [ ]:
#Genero las recomendaciones.
_, indices = knn_model.kneighbors(tfidf_matrix[106106])
recommendations = local_categories['business_id'].iloc[indices[0][1:]]  # Excluye el propio restaurante

In [ ]:
print(local_categories.iloc[106106])

In [ ]:
print(recommendations)

In [ ]:
local_categories[local_categories['business_id']=='0x88d905f9b0dccd0d:0x189d7a76056de69a']